In [ ]:
from google.colab import files
uploaded = files.upload()


Saving image_class_labels.txt to image_class_labels.txt
Saving images.txt to images.txt
Saving train_test_split.txt to train_test_split.txt


In [ ]:
import pandas as pd

# Set base path for uploaded files
base_path = "/content/"

# Load metadata files
images_df = pd.read_csv(base_path + "images.txt", sep=" ", header=None, names=["image_id", "file_path"])
labels_df = pd.read_csv(base_path + "image_class_labels.txt", sep=" ", header=None, names=["image_id", "class_id"])
split_df = pd.read_csv(base_path + "train_test_split.txt", sep=" ", header=None, names=["image_id", "is_train"])

# Merge into one master DataFrame
metadata_df = images_df.merge(labels_df, on="image_id")
metadata_df = metadata_df.merge(split_df, on="image_id")

# Preview result
metadata_df.head()


,image_id,file_path,class_id,is_train
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# Step 2: Set your base path inside Google Drive
# (You said you uploaded to: My Drive > Colab Notebooks > CUB_200_2011)
base_dir = "/content/drive/MyDrive/Colab Notebooks/CUB_200_2011"
image_dir = f"{base_dir}/images"  # folder containing all class folders and images
metadata_dir = base_dir           # where all .txt files are


# Step 5: Add full image path
metadata_df["full_path"] = metadata_df["file_path"].apply(lambda x: f"{image_dir}/{x}")

# Step 6: Split into train and test sets
train_df = metadata_df[metadata_df["is_train"] == 1].reset_index(drop=True)
test_df = metadata_df[metadata_df["is_train"] == 0].reset_index(drop=True)

# Optional: check a few rows
print("Train set size:", len(train_df))
print("Test set size:", len(test_df))
metadata_df.head()


Train set size: 5994
Test set size: 5794


,image_id,file_path,class_id,is_train,full_path
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0,/content/drive/MyDrive/Colab Notebooks/CUB_200...
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1,/content/drive/MyDrive/Colab Notebooks/CUB_200...
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0,/content/drive/MyDrive/Colab Notebooks/CUB_200...
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1,/content/drive/MyDrive/Colab Notebooks/CUB_200...
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1,/content/drive/MyDrive/Colab Notebooks/CUB_200...


In [ ]:
import tensorflow as tf


# Step 2A: Image loading and preprocessing function
def load_and_preprocess_image(path, label):
    # ✅ This line loads the image from Google Drive
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)

    # Resize (shorter side = 256) — using resize_with_pad to preserve aspect
    image = tf.image.resize_with_pad(image, target_height=256, target_width=256)

    # Random crop to 224×224
    image = tf.image.random_crop(image, size=[224, 224, 3])

    # Data augmentation (to match paper)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)

    # Normalize with ImageNet mean/std
    image = tf.cast(image, tf.float32) / 255.0
    image = (image - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]

    return image, label



In [ ]:
def load_and_preprocess_image_val(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)

    # Resize and center crop
    image = tf.image.resize_with_pad(image, 256, 256)
    image = tf.image.central_crop(image, central_fraction=0.875)  # ~224/256 = 0.875

    # Normalize
    image = tf.cast(image, tf.float32) / 255.0
    image = (image - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]

    return image, label



In [ ]:
# Convert Pandas train_df into TensorFlow dataset
# Step 2B: Create TensorFlow dataset from train_df
train_paths = train_df["full_path"].tolist()
train_labels = train_df["class_id"].astype(int) - 1  # shift to 0-based index

train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Add caching and prefetching to optimize performance
train_dataset = train_dataset.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
val_paths = test_df["full_path"].tolist()
val_labels = test_df["class_id"].astype(int) - 1

val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image_val, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
# Step 4: Load and modify pretrained ResNet-101-V2
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras import layers, models

# Load base model
base_model = ResNet101V2(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling='avg')
x = layers.Dense(200, activation='softmax')(base_model.output)
model = models.Model(inputs=base_model.input, outputs=x)

171317808/171317808 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Step 5: Compile the model with optimizer and hyperparameters
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# test small batch first

training_metrics_small_batch  = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1,
    steps_per_epoch=3,        # Only 3 training batches
    validation_steps=1        # Only 1 validation batch
)


3/3 ━━━━━━━━━━━━━━━━━━━━ 336s 55s/step - accuracy: 0.0000e+00 - loss: 5.4413 - val_accuracy: 0.0000e+00 - val_loss: 10.8287


In [ ]:
# All images loading (don't run below code? prefer using subset ?)
'''
# Step 6: Train the model (triggers actual image loading)
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=100)